 ---- TODO ---- DELETE BEFORE SUBMITING
  Formatting Instructions for Notebook
1. Structure your notebook following the same style used in previous assignments.
2. Use Markdown headings to clearly separate each major section (e.g., Load Data,
Train Classifier etc.).
3. Provide brief explanations in text cells to describe what each section does.
4. Add comments in your code to explain important implementation details.
5. Use separate code cells for different datasets and different parts of the process, such
as pre-processing, training, and evaluation.
6. Do not write all your code or all tests for different datasets in a single code cell.

IMPORTS

In [1]:
# imports
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

TREE NODE
    This class models an individual node in the decision tree that tracks
        - If the node is a leaf
            - If it is a leaf it stores the class label
        - Else it stores refrences to the left and right children

In [2]:
class Node:
    def __init__ (self, is_leaf, class_label, left, right):
        self.is_leaf     = is_leaf
        self.class_label = class_label
        self.left        = left
        self.right       = right

GINI IMPURITY

In [3]:
def gini_impurity(series):
    size = len(series)
    series_counts = series.value_counts()

    impurity = 1
    for count in series_counts: 
        proportion = count/size
        impurity -= proportion**2

    return impurity
        

CREATE ALL BOUNDARIES - create all possible partitions of the the classes

In [4]:
# this function creates all possible boundary sets by outputting a 2d list. each inside list contains a unique boundary
def all_boundaries(classes):
    # this is assuming the the classes input will be a series
    classes = set(classes)
    boundaries     = []

    # Single classes
    # item in this for loop referes to class within set classes but python does like me calling a variable class
    for single in classes:
        other  = classes - {single}
        boundaries.append([{single}, other])
            
    class_list = sorted(classes)
        
    # Paired classes
    for r in range (len(class_list)):
        for c in range (r + 1, len(class_list)):
            pair  = {class_list[r],  class_list[c]}
            other = classes - pair
            boundaries.append([pair, other])
    return boundaries

SCORE BOUNDARY - scores a given boundary using gini impurity. lower scores indicate a better boundary

In [5]:
# this function will take in the class and data and score the boundary
def boundary_score(X, y, boundary, function):

    # boundary is a list containing 2 sets - the left set (positive) and right set (negative)
    positive_set = boundary[0]
    y_binary = []

    # turn the boundary from multiple classes to two classes
    for item in y:
        if item in positive_set:
            y_binary.append(1)
        else:
            y_binary.append(-1)
    
    SVM = SVC(kernel = function)
    SVM.fit(X, y_binary)

    predictions = SVM.predict(X)

    # positive labelings, only count when the prediction is 1
    pos = y[predictions == 1]
    # negative, only when prediction is -1
    neg = y[predictions == -1]

    # computes the impurity of positive and negative
    gini_pos = gini_impurity(pos)
    gini_neg = gini_impurity(neg)
    
    n = len(y_train)
    weight_pos = len(pos) / n
    weight_neg = len(neg) / n
    # weights the impurity so uneven splits don't mess up the impurity rating
    weighted_gini = weight_pos * gini_pos + weight_neg * gini_neg
    
    # this will return gini score
    return weighted_gini

BUILD TREE

In [6]:
def build_tree(X, y, depth, function):
    print(depth)
    print("\n")
    # check for pure node (all points belong to one class) 
    if (len(set(y)) == 1):
        return Node(True, y, None, None)

    # check if the support of the branch is low < 10
    if (len(y) < 10):
        return Node(True, y, None, None)

    # check if max depth has been reached
    if (depth == MAX_TREE_HEIGHT):
        return Node(True, y, None, None)

    # lowest score is the best score
    # best score can be 1 because gini only goes up to 0.5
    best_score = 1
    best_boundary = None
    
    # create all possible boundaries
    boundaries = all_boundaries(y)
    
    # find the best boundary by searching for the lowest possible gini impurity
    for b in boundaries:
        b_score = boundary_score(X,y,b,function)
        if (b_score < best_score):
            best_score = b_score
            best_boundary = b

    # probably should check if the node should be a leaf, i would imagine that would be the case if there was no good split
    
    left_classes, right_classes = best_boundary

    # split the data by the boundary
    left_y = y[y.isin(left_classes)]
    right_y = y[y.isin(right_classes)]

    
    left_x = X[y.isin(left_classes)]
    right_x = X[y.isin(right_classes)]

    # create left child with y only including left classes
    left_child = build_tree(left_x, left_y, depth + 1, function)
    # create right child with y only including right classes
    right_child = build_tree(right_x, right_y, depth + 1, function)

    # create the node - class label should be something
    return Node(False, None, left_child, right_child)

LOAD DATA

In [7]:
data1_train = pd.read_csv("data/Data1Train.csv")
data2_train = pd.read_csv("data/Data2Train.csv")
data3_train = pd.read_csv("data/Data3Train.csv")
data4_train = pd.read_csv("data/Data4Train.csv")

# display data set 1
data1_train.head()

,Class,x,y
0,4,0.803929,0.230782
1,1,0.242077,0.840158
2,4,0.687047,0.280138
3,1,0.086210,0.765054
4,2,0.589790,0.809382


Analyzing data1

In [8]:
MAX_TREE_HEIGHT = math.ceil(math.log2(4)) + 1
random_state = 1234
X = data1_train[['x', 'y']]
y = data1_train['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = random_state)

# TODO store the tree in a way that supports traversal and testing
build_tree(X, y, 0, "linear")

0


1


1


2


2





KeyboardInterrupt



Analyzing data2

In [ ]:
MAX_TREE_HEIGHT = math.ceil(math.log2(4)) + 1
random_state = 1234
X = data2_train[['x', 'y']]
y = data2_train['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = random_state)

# TODO store the tree in a way that supports traversal and testing
build_tree(X, y, 0, "linear")

Analyzing data3

In [ ]:
MAX_TREE_HEIGHT = math.ceil(math.log2(3)) + 1
random_state = 1234
X = data2_train[['x', 'y']]
y = data2_train['Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = random_state)

# TODO store the tree in a way that supports traversal and testing
build_tree(X, y, 0, "linear")

Analyzing data4

In [ ]:
MAX_TREE_HEIGHT = math.ceil(math.log2(4)) + 1
random_state = 1234
X = data4_train[['x', 'y']]
y = data4_train['Class']

# Data set 4 is nonlinear so scale x
scaler = standarScaler()
X_scale = scaler.fit_transform(X)
# TODO store the tree in a way that supports traversal and testing
build_tree(X_scale, y, 0, "rbf")

THING 5